## _*Tapering an Operator*_

This notebook demonstrates how symmetries can be taken advantage of to reduce the size (number of qubits needed) for an Operator when using Qiskit Chemistry.

This notebook has been written to use the PYSCF chemistry driver.

In [1]:
# import common packages
import itertools
import logging

import numpy as np

from qiskit import BasicAer

from qiskit.aqua import QuantumInstance
from qiskit.aqua.operators import Z2Symmetries, WeightedPauliOperator
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA

from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType 
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

Couldn't find cython int routine
Couldn't find cython int routine


In [2]:
# using driver to get fermionic Hamiltonian
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

In [3]:
core = Hamiltonian(transformation=TransformationType.FULL, qubit_mapping=QubitMappingType.PARITY, 
                   two_qubit_reduction=True, freeze_core=True)
qubit_op, _ = core.run(molecule)

print("Originally requires {} qubits".format(qubit_op.num_qubits))
print(qubit_op)

Originally requires 8 qubits
Electronic Hamiltonian: Representation: paulis, qubits: 8, size: 276


Find the symmetries of the qubit operator

In [4]:
z2_symmetries = Z2Symmetries.find_Z2_symmetries(qubit_op)
print('Z2 symmetries found:')
for symm in z2_symmetries.symmetries:
    print(symm.to_label())
print('single qubit operators found:')
for sq in z2_symmetries.sq_paulis:
    print(sq.to_label())
print('cliffords found:')
for clifford in z2_symmetries.cliffords:
    print(clifford.print_details())
print('single-qubit list: {}'.format(z2_symmetries.sq_list))

Z2 symmetries found:
ZIZIZIZI
ZZIIZZII
single qubit operators found:
IIIIIIXI
IIIIIXII
cliffords found:
ZIZIZIZI	(0.7071067811865475+0j)
IIIIIIXI	(0.7071067811865475+0j)

ZZIIZZII	(0.7071067811865475+0j)
IIIIIXII	(0.7071067811865475+0j)

single-qubit list: [1, 2]


Use the found symmetries, single qubit operators, and cliffords to taper qubits from the original qubit operator. For each Z2 symmetry one can taper one qubit. However, different tapered operators can be built, corresponding to different symmetry sectors. 

In [5]:
tapered_ops = z2_symmetries.taper(qubit_op)
for tapered_op in tapered_ops:
    print("Number of qubits of tapered qubit operator: {}".format(tapered_op.num_qubits))

Number of qubits of tapered qubit operator: 6
Number of qubits of tapered qubit operator: 6
Number of qubits of tapered qubit operator: 6
Number of qubits of tapered qubit operator: 6


The user has to specify the symmetry sector he is interested in. Since we are interested in finding the ground state here, let us get the original ground state energy as a reference.

In [6]:
ee = NumPyMinimumEigensolver(qubit_op)
result = core.process_algorithm_result(ee.run())
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.874303870396
  - computed part:      -1.078084301625
  - frozen energy part: -7.796219568771
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.992207270475
> Total ground state energy (Hartree): -7.882096599921


Now, let us iterate through all tapered qubit operators to find out the one whose ground state energy matches the original (un-tapered) one.

In [7]:
smallest_eig_value = 99999999999999
smallest_idx = -1
for idx in range(len(tapered_ops)):
    ee = NumPyMinimumEigensolver(tapered_ops[idx])
    curr_value = ee.run().eigenvalue.real
    if curr_value < smallest_eig_value:
        smallest_eig_value = curr_value
        smallest_idx = idx
    print("Lowest eigenvalue of the {}-th tapered operator (computed part) is {:.12f}".format(idx, curr_value))
    
the_tapered_op = tapered_ops[smallest_idx]
the_coeff = tapered_ops[smallest_idx].z2_symmetries.tapering_values
print("The {}-th tapered operator matches original ground state energy, with corresponding symmetry sector of {}".format(smallest_idx, the_coeff))

Lowest eigenvalue of the 0-th tapered operator (computed part) is -1.078084301625
Lowest eigenvalue of the 1-th tapered operator (computed part) is -0.509523578167
Lowest eigenvalue of the 2-th tapered operator (computed part) is -0.912078232998
Lowest eigenvalue of the 3-th tapered operator (computed part) is -0.912078232998
The 0-th tapered operator matches original ground state energy, with corresponding symmetry sector of [1, 1]


Alternatively, one can run multiple VQE instances to find the lowest eigenvalue sector. 
Here we just validate that `the_tapered_op` reach the smallest eigenvalue in one VQE execution with the UCCSD variational form, modified to take into account of the tapered symmetries.

In [8]:
# setup initial state
init_state = HartreeFock(num_orbitals=core._molecule_info['num_orbitals'],
                    qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction,
                    num_particles=core._molecule_info['num_particles'], sq_list=the_tapered_op.z2_symmetries.sq_list)

# setup variationl form
var_form = UCCSD(num_orbitals=core._molecule_info['num_orbitals'], 
                 num_particles=core._molecule_info['num_particles'],
                 active_occupied=None, active_unoccupied=None, initial_state=init_state,
                 qubit_mapping=core._qubit_mapping, two_qubit_reduction=core._two_qubit_reduction, 
                 num_time_slices=1, z2_symmetries=the_tapered_op.z2_symmetries)

# setup optimizer
optimizer = COBYLA(maxiter=1000)

# set vqe
algo = VQE(the_tapered_op, var_form, optimizer)

# setup backend
backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend)

In [9]:
algo_result = algo.run(quantum_instance)

In [10]:
result = core.process_algorithm_result(algo_result)
print(result)

print("The parameters for UCCSD are:\n{}".format(algo_result.optimal_point))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.874303841496
  - computed part:      -1.078084272725
  - frozen energy part: -7.796219568771
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.992207270475
> Total ground state energy (Hartree): -7.882096571021
The parameters for UCCSD are:
[ 0.03803094  0.00360661  0.0382837   0.00369849 -0.03608325  0.05942172
 -0.02729715 -0.02732059  0.05965191 -0.11498381]
